# Introducción

En el siguiente proyecto estaré desarrollando un modelo de aprendizaje para un banco que está experimentando una creciente tasa de abandono por parte de sus clientes cada mes.  Para el banco es más económico salvar a los clientes que permanecen, que atrer a nuevos. 

Mi objetivo es crear un modelo de clasificación que me permita predecir exitosamente si un cliente abandonará el banco pronto. Para cumplir con esa tarea, estaré trabajando con la base de datos proporcionada por el banco, la cual me permitirá evaluar el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Mi enfoque en cuanto a las predicciones de si un cliente abandonará o no, será disminuir el máximo posible de falsos negativos y falsos positivos; para lo cual me concentraré en la media armónica de las métricas de precisión y sensibilidad.

#### Tabla de contenido:

1. Inicialización: importar las librerías.
  - 1.2. Cargar y preparar los datos. 
2. Codificación de los datos categóricos:
  - 2.2. Codificación OHE, segmentación de conjuntos de datos y estandarización de características numéricas para modelos de regresión.
  - 2.3. Codificación de etiquetas, segmentación de conjuntos de datos y estandarización de características numéricas  para modelos basados en árboles.
3. Entrenar los modelos.
  - 3.2. Examinar el equilibrio de clases.
4. Elegir un modelo y comprobar su calidad con el conjunto de prueba.
5. Prueba de cordura en el modelo elegido.
6. Conclusiones generales.

# 1. Inicialización

In [1]:
#importar librerías
import pandas as pd
from sklearn.metrics import recall_score, precision_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.utils import shuffle
import matplotlib as plt

#librerias para los modelos 
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## 1.2. Cargar y preparar los datos 

In [2]:
#cargar el dataset en un dataframe
data = pd.read_csv('dataset/Churn.csv')

In [3]:
#visualizar información general del dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
#visualizar los datos
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


En cuanto a los tipos de datos de cada columna, parecen correctos para cada categoría correspondiente. Sin embargo, para mi tarea de clasificación debo codificar más adelante algunas columnas categóricas; estas columnas serán: 'Surname', 'Geography' y 'Gender'.

In [5]:
#verificar los duplicados en el dataframe
data.duplicated().sum()

0

In [6]:
#verificar datos ausentes
data.isnull().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

He encontrado algunos datos ausentes en la columna 'Tenure'. Esta columna contiene el período durante el cual ha madurado el depósito a plazo fijo de un cliente (años).
En vista de que esa información pudiera ser importante para que el modelo pueda predecir si un cliente abandonará el banco, procederé a rellenar los datos ausentes con el período mediano, agrupando por país, género y edad.

In [7]:
#rellenar los datos ausentes en la columna 'Tenure'
data['Tenure'] = data['Tenure'].fillna(data.groupby(['Geography', 'Gender', 'Age'])['Tenure'].transform('median'))

#verificar nuevamente los datos ausentes
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             3
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [28]:
#Eliminar columnas innecesarias del DataFrame
set_data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Preparé un DataFrame, eliminando los datos ausentes y las columnas innecesarias para realizar las predicciones.
Los datos ya están preparados para pasar a la segmentación de los conjuntos de entrenamiento, validación y prueba.

# 2. Codificación de columnas categóricas

### 2.2. Codificación OHE de columnas categóricas para los modelos de regresión
### y estandarización de características numéricas

In [21]:
#codificación OHE para las regresiones
data_ohe = pd.get_dummies(data, drop_first=True)

#definir el objetivo y las features
target_ohe = data_ohe['Exited']
features_ohe = data_ohe.drop(['Exited'], axis=1)

#segmentación de los datos para conjuntos de entrenamiento y validación ohe
features_train_ohe, features_valid_ohe, target_train_ohe, target_valid_ohe = train_test_split(features_ohe, target_ohe, test_size=0.40, random_state=12345)

#segmentacion del conjunto de validacion para conjunto de prueba ohe
features_valid_ohe, features_test_ohe, target_valid_ohe, target_test_ohe = train_test_split(features_valid_ohe, target_valid_ohe, test_size=0.40, random_state=12345)

#verificando los tamaños de los conjuntos 
print('Tamaños de los conjuntos de datos:')
print('features train ohe:', features_train_ohe.shape)
print('target train ohe:', target_train_ohe.shape)
print()
print('Conjuntos de validación:')
print('features valid ohe:', features_valid_ohe.shape)
print('target_valid ohe:', target_valid_ohe.shape)
print()
print('Conjuntos de prueba:')
print('features test ohe:', features_test_ohe.shape)
print('target test ohe:', target_test_ohe.shape)

data_ohe

Tamaños de los conjuntos de datos:
features train ohe: (6000, 2944)
target train ohe: (6000,)

Conjuntos de validación:
features valid ohe: (2400, 2944)
target_valid ohe: (2400,)

Conjuntos de prueba:
features test ohe: (1600, 2944)
target test ohe: (1600,)


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,...,Surname_Zotova,Surname_Zox,Surname_Zubarev,Surname_Zubareva,Surname_Zuev,Surname_Zuyev,Surname_Zuyeva,Geography_Germany,Geography_Spain,Gender_Male
0,1,15634602,619,42,2.0,0.00,1,1,1,101348.88,...,False,False,False,False,False,False,False,False,False,False
1,2,15647311,608,41,1.0,83807.86,1,0,1,112542.58,...,False,False,False,False,False,False,False,False,True,False
2,3,15619304,502,42,8.0,159660.80,3,1,0,113931.57,...,False,False,False,False,False,False,False,False,False,False
3,4,15701354,699,39,1.0,0.00,2,0,0,93826.63,...,False,False,False,False,False,False,False,False,False,False
4,5,15737888,850,43,2.0,125510.82,1,1,1,79084.10,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,771,39,5.0,0.00,2,1,0,96270.64,...,False,False,False,False,False,False,False,False,False,True
9996,9997,15569892,516,35,10.0,57369.61,1,1,1,101699.77,...,False,False,False,False,False,False,False,False,False,True
9997,9998,15584532,709,36,7.0,0.00,1,0,1,42085.58,...,False,False,False,False,False,False,False,False,False,False
9998,9999,15682355,772,42,3.0,75075.31,2,1,0,92888.52,...,False,False,False,False,False,False,False,True,False,True


In [31]:
#estadarizar las características numéricas
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train_ohe[numeric])
features_train_ohe[numeric] = scaler.transform(features_train_ohe[numeric])
features_valid_ohe[numeric] = scaler.transform(features_valid_ohe[numeric])
features_test_ohe[numeric] = scaler.transform(features_test_ohe[numeric])



Para los modelos de regresión que entrenaré más adelante, he codificado las columnas categóricas del DataFrame utilizando una codificación One-Hot y posteriormente estandaricé las características numéricas para que mis modelos las consideren a todas igual de importantes.

### 2.3. Codificación de etiquetas para modelos basados en árboles
### y estandarización de características numéricas

In [32]:
#ordinal encoder para modelos basados en árboles
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(data), columns=data.columns)

#definir el objetivo y las features 
target_ordinal = data_ordinal['Exited']
features_ordinal = data_ordinal.drop(['Exited'], axis=1)

#segmentar los datos en conjuntos de entrenamiento y validcación
features_train_ordinal, features_valid_ordinal, target_train_ordinal, target_valid_ordinal = train_test_split(features_ordinal, target_ordinal, test_size=0.40, random_state=12345)

#segmentar el conjunto de validación en un subconjunto de prueba
features_valid_ordinal, features_test_ordinal, target_valid_ordinal, target_test_ordinal = train_test_split(features_valid_ordinal, target_valid_ordinal, test_size=0.20, random_state=12345)


print('Tamaños de los conjuntos de datos:')
print('features train ordinal:' , features_train_ordinal.shape)
print('target train ordinal:' , target_train_ordinal.shape)
print()
print('features valid ordinal:' , features_valid_ordinal.shape)
print('target valid ordinal:' , target_valid_ordinal.shape)
print()
print('features test ordinal:', features_test_ordinal.shape)
print('target test ordinal:', target_test_ordinal.shape)

Tamaños de los conjuntos de datos:
features train ordinal: (6000, 13)
target train ordinal: (6000,)

features valid ordinal: (3200, 13)
target valid ordinal: (3200,)

features test ordinal: (800, 13)
target test ordinal: (800,)


In [33]:
#estandarizar las características para los modelos basados en árboles
scaler_ordinal = StandardScaler()
scaler_ordinal.fit(features_train_ordinal)

features_train_ordinal = scaler_ordinal.transform(features_train_ordinal)
features_valid_ordinal = scaler_ordinal.transform(features_valid_ordinal)
features_test_ordinal = scaler_ordinal.transform(features_test_ordinal)


Para los modelos basados en árboles que voy a entrenar, decidí codificar las columnas categóricas utilizando la codificación de etiquetas, más especificamente, el OrdinalEncoder y proseguí estandarizando las características numéricas para asignarles igual importancia.

## 3. Entrenar modelos

### 3.2. Modelo basado en árboles de decisión

In [34]:
#conjuntos de entrenamiento 
features_train_ordinal
target_train_ordinal

#conjuntos de validación
features_valid_ordinal
target_valid_ordinal

#conjuntos de prueba
features_test_ordinal
target_test_ordinal


best_model = None
best_depth = 0
best_result = 0.59

for depth in range (1,10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train_ordinal, target_train_ordinal)
    
    predicted_valid = model.predict(features_valid_ordinal)
    result = f1_score(target_valid_ordinal, predicted_valid)

    #if result > best_result:
        #best_model = model
        #best_result = result
        #best_depth = depth
